# ASL Project

In [108]:
#Import libraries
import numpy as np
import random

# Imports to view data
import cv2
import pandas as pd
from glob import glob

# Visualization
from keras.utils import print_summary
from matplotlib import pyplot as plt

#ML libraries
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint  
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [45]:
#Directory paths
TRAIN_DIR = "../Dataset/asl_alphabet_train/asl_alphabet_train"
TEST_DIR = "../Dataset/asl_alphabet_test"
MODEL_DIR = './Model'
MODEL_PATH = MODEL_DIR+"/Model.h5"
MODEL_WEIGHT_PATH = MODEL_DIR+"/Model_Weight.h5"

In [17]:
# Set global variables
TARGET_SIZE = (64, 64)
TARGET_DIMS = (64, 64, 3) # add channel for RGB
CLASSES = 29
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 64

## Data Augmentation

In [52]:
#Load Train dataset
train_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

validation_image_generator = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    validation_split=VALIDATION_SPLIT
)

train_generator = train_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, shuffle=True, subset="training")
val_generator = validation_image_generator.flow_from_directory(TRAIN_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


## Model

In [31]:
#Define Model

def model_build():
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=TARGET_DIMS,padding='same'))
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    model.add(MaxPooling2D((2, 2),padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128, activation='linear'))
    model.add(Dropout(0.2))                  
    model.add(Dense(CLASSES, activation='softmax'))

    model.summary()
    
    return model

In [32]:
model = model_build()
#Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.save(MODEL_PATH)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 8, 8, 64)          73792     
__________

In [40]:
#Checkpointer to save the best models
checkpointer = ModelCheckpoint(filepath=MODEL_WEIGHT_PATH, 
                               verbose=1, save_best_only=True)

steps_per_epoch = int( np.ceil(len(train_generator)*2 / BATCH_SIZE) )
validation_steps = int( np.ceil(len(val_generator)*2 / BATCH_SIZE) )

model.fit_generator(train_generator, validation_data=val_generator, 
                    steps_per_epoch =  steps_per_epoch,
                    validation_steps = validation_steps,
                    epochs=10, callbacks=[checkpointer], verbose=1)

Epoch 1/10
39/39 [==============================] - 36s 924ms/step - loss: 2.8866 - acc: 0.1470 - val_loss: 2.8089 - val_acc: 0.1469

Epoch 00001: val_loss improved from inf to 2.80894, saving model to ./Model/Model_Weight.h5
Epoch 2/10
39/39 [==============================] - 37s 957ms/step - loss: 2.7279 - acc: 0.1795 - val_loss: 2.4601 - val_acc: 0.2687

Epoch 00002: val_loss improved from 2.80894 to 2.46009, saving model to ./Model/Model_Weight.h5
Epoch 3/10
39/39 [==============================] - 38s 980ms/step - loss: 2.5966 - acc: 0.2135 - val_loss: 2.6455 - val_acc: 0.2219

Epoch 00003: val_loss did not improve from 2.46009
Epoch 4/10
39/39 [==============================] - 38s 965ms/step - loss: 2.4975 - acc: 0.2436 - val_loss: 2.1808 - val_acc: 0.2875

Epoch 00004: val_loss improved from 2.46009 to 2.18079, saving model to ./Model/Model_Weight.h5
Epoch 5/10
39/39 [==============================] - 39s 997ms/step - loss: 2.3787 - acc: 0.2644 - val_loss: 2.3858 - val_acc: 0.2

In [97]:
#Predict on validation dataset
predictions = model.predict_generator(val_generator, steps=1)        
predictions = np.argmax(predictions, axis=-1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

loss, acc = model.evaluate_generator(val_generator, steps=1, verbose=0)

print(loss,",",acc)

['nothing', 'E', 'Q', 'Q', 'Q', 'C', 'del', 'D', 'L', 'P', 'O', 'E', 'D', 'T', 'M', 'C', 'X', 'L', 'J', 'B', 'R', 'T', 'W', 'H', 'L', 'V', 'S', 'T', 'V', 'C', 'W', 'P', 'del', 'D', 'nothing', 'T', 'P', 'S', 'Z', 'E', 'O', 'S', 'D', 'T', 'del', 'X', 'E', 'F', 'nothing', 'Y', 'nothing', 'F', 'W', 'C', 'J', 'F', 'T', 'R', 'D', 'E', 'F', 'Q', 'T', 'E'] 64
1.8403854370117188 , 0.421875


In [113]:
test_image_generator = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True,
)

test_generator = test_image_generator.flow_from_directory(TEST_DIR, target_size=TARGET_SIZE, batch_size=28, shuffle=False, 
    class_mode='categorical')
print(test_generator.classes)

Found 28 images belonging to 1 classes.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [114]:
#Predict
test_generator.reset()
predictions = model.predict_generator(test_generator, steps=1)
predictions = np.argmax(predictions, axis=1) #multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v

predictions = [label_map[k] for k in predictions]

print(predictions, len(predictions))

['M', 'B', 'C', 'D', 'E', 'F', 'P', 'P', 'R', 'J', 'V', 'L', 'M', 'J', 'O', 'P', 'Q', 'R', 'S', 'J', 'U', 'V', 'W', 'X', 'T', 'Y', 'nothing', 'space'] 28


In [115]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
print(results)

                              Filename Predictions
0         asl_alphabet_test\A_test.jpg           M
1         asl_alphabet_test\B_test.jpg           B
2         asl_alphabet_test\C_test.jpg           C
3         asl_alphabet_test\D_test.jpg           D
4         asl_alphabet_test\E_test.jpg           E
5         asl_alphabet_test\F_test.jpg           F
6         asl_alphabet_test\G_test.jpg           P
7         asl_alphabet_test\H_test.jpg           P
8         asl_alphabet_test\I_test.jpg           R
9         asl_alphabet_test\J_test.jpg           J
10        asl_alphabet_test\K_test.jpg           V
11        asl_alphabet_test\L_test.jpg           L
12        asl_alphabet_test\M_test.jpg           M
13        asl_alphabet_test\N_test.jpg           J
14        asl_alphabet_test\O_test.jpg           O
15        asl_alphabet_test\P_test.jpg           P
16        asl_alphabet_test\Q_test.jpg           Q
17        asl_alphabet_test\R_test.jpg           R
18        asl_alphabet_test\S_t

In [136]:
count = 0
for file,prediction in zip(filenames,predictions):
    #print(file,prediction)
    if(prediction+'_test' in file):
        count+=1
        
print("accuracy",count/len(filenames)*100)

accuracy 67.85714285714286
